In [43]:
from functools import lru_cache
from itertools import combinations

def fixed_parts_graphs(n0, n1):
    if n0==0:
        yield biflag(n1, 0)
        return
    if n1==0:
        yield biflag(n0, n0)
        return
    from sage.features.nauty import NautyExecutable
    import subprocess, select
    import shlex
    directg_path = NautyExecutable("genbg").absolute_filename()
    options="-l {} {}".format(n0, n1)
    sub = subprocess.Popen(
        shlex.quote(directg_path) + ' {0}'.format(options),
        shell=True,
        stdout=subprocess.PIPE,
        stdin=subprocess.PIPE,
        stderr=subprocess.PIPE,
        encoding='latin-1'
    )
    sub.stdin.close()
    if n0!=n1:
        for line in sub.stdout:
            g = Graph(line, format="graph6")
            yield biflag(n0+n1, n0, edges=tuple(g.edges(labels=False)))
    else:
        uniques = []
        for line in sub.stdout:
            g = Graph(line, format="graph6")
            can = tuple(g.canonical_label().edges(labels=None))
            if can not in uniques:
                uniques.append(can)
                yield biflag(n0+n1, n0, edges=tuple(g.edges(labels=False)))
    sub.wait()

@lru_cache(maxsize=None)
def bipartite_graphs_all(n):
    ret = []
    for split in range(n//2 + 1):
        for xx in fixed_parts_graphs(split, n-split):
            ret.append(xx)
    return tuple(ret)

@lru_cache(maxsize=None)
def bipartite_graphs_split(n, s):
    return tuple(fixed_parts_graphs(s, n-s))

class biflag:
    def __init__(self, n, split, edges=None, tps=None):
        self.n = n
        self.edges = list(edges) if edges!=None else []
        self.tps = list(tps) if tps!=None else []
        self.split = split
        self._unique = None
        if split>n//2:
            self.edges = [(n-1-p0, n-1-p1) for p0, p1 in self.edges]
            self.tps = [n-1-ii for ii in self.tps]
            self.split = n-split

    def __str__(self):
        return "Biflag on {} split {} tps {} edges [{}]".format( \
            self.n, self.split, self.tps, " ".join(["{}{}".format(ee[0], ee[1]) for ee in self.edges]))

    __repr__ = __str__
    
    def subflag(self, n=None, tps=None):
        if tps==None:
            tps = self.tps
        if n==None:
            n = list(range(self.n))
        else:
            n = list(set(list(n)+list(tps)))
        if len(n)==self.n and tps==self.tps:
            return self
        n.sort()
        new_split = len([xx for xx in n if xx<self.split])
        new_edges = [(n.index(p0), n.index(p1)) for p0, p1 in self.edges if (p0 in n and p1 in n)]
        new_tps = [n.index(ii) for ii in tps]
        return biflag(len(n), new_split, edges=new_edges, tps=new_tps)
    
    def unique(self):
        if self._unique != None:
            return self._unique
        else:
            nsum = self.n
            V = list(range(nsum+2))
            E = self.edges + [(nsum, ii) for ii in range(self.split)] + [(nsum+1, ii) for ii in range(self.split, self.n)]
            P = [[ii] for ii in self.tps] + [[jj for jj in range(nsum) if jj not in self.tps]] + [[nsum, nsum+1]]
            g = Graph([V, E], format="vertices_and_edges")
            can = g.canonical_label(partition=P)
            unique = (self.n, self.split, len(self.tps), tuple(can.edges(labels=None, sort=True)))
            self._unique = unique
            return unique

    def __hash__(self):
        return hash(self.unique())
    
    def __eq__(self, other):
        if self.n!=other.n or self.split!=other.split or len(self.edges)!=len(other.edges) or len(self.tps)!=len(other.tps):
            return False
        return self.unique()==other.unique()

#small size=n0, large size = n1
@lru_cache(maxsize=None)
def cont_matrix(n0, s0, n1, s1):
    ls0 = list(bipartite_graphs_split(n0, s0))
    ls1 = list(bipartite_graphs_split(n1, s1))
    if n0>n1 or s0>s1 or (n0-s0)>(n1-s1):
        return matrix(QQ, len(ls0), len(ls1), sparse=True)
    if n0==n1 and s0==s1:
        return matrix(QQ, len(ls0), len(ls1), 1, sparse=True)
    data = {}
    for iyy, yy in enumerate(ls1):
        for pts in combinations(range(s1), s0):
            for ptr in combinations(range(s1, n1), n0-s0):
                subf = yy.subflag(n=(list(pts) + list(ptr)), tps=[])
                try:
                    ixx = ls0.index(subf)
                except:
                    print("Subflag is missing {} original {} points {}, {}".format(subf, yy, pts, ptr))
                data[(ixx, iyy)] = 1
    return matrix(QQ, len(ls0), len(ls1), data, sparse=True)

In [45]:
cont_matrix(3, 2, 5, 3)

[1 1 1 1 1 1 1 0 1 1 0 0 0]
[0 1 1 1 1 1 1 1 1 1 1 1 0]
[0 0 0 1 0 0 1 0 1 1 1 1 1]